pneumonia detection using transfer learning in a easier way 


In [1]:
# import the libraries as shown below

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

assigning path to data

In [2]:
#resize the images
#provide path for test and train data

IMAGE_SIZE =[224,224]
train_path = '../input/chest-xray-pneumonia/chest_xray/train'
test_path = '../input/chest-xray-pneumonia/chest_xray/test'

selecting relevant model

In [3]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 3s 0us/step


In [4]:
# its important that we don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [5]:
  # useful for getting number of output classes
folders = glob('../input/chest-xray-pneumonia/chest_xray/train/*')

In [6]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

In [7]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [8]:
# for viewing the structure of the model

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

optimization


In [9]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

importing images from dataset

In [10]:

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('../input/chest-xray-pneumonia/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 5216 images belonging to 2 classes.


In [12]:
test_set = test_datagen.flow_from_directory('../input/chest-xray-pneumonia/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [13]:
len(training_set)


163

In [14]:
len(test_set)

20

training the data

In [15]:
#fit the model
#change the parameters according to your needs to get a better accuracy


r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch= len(training_set),
  validation_steps= len(test_set)
)

Epoch 1/5
163/163 [==============================] - 147s 905ms/step - loss: 0.1950 - accuracy: 0.9243 - val_loss: 0.2452 - val_accuracy: 0.9167
Epoch 2/5
163/163 [==============================] - 106s 650ms/step - loss: 0.1424 - accuracy: 0.9525 - val_loss: 0.3709 - val_accuracy: 0.9006
Epoch 3/5
163/163 [==============================] - 106s 652ms/step - loss: 0.0893 - accuracy: 0.9676 - val_loss: 0.2450 - val_accuracy: 0.9199
Epoch 4/5
163/163 [==============================] - 106s 652ms/step - loss: 0.0965 - accuracy: 0.9640 - val_loss: 0.2574 - val_accuracy: 0.9231
Epoch 5/5
163/163 [==============================] - 107s 655ms/step - loss: 0.0844 - accuracy: 0.9718 - val_loss: 0.3006 - val_accuracy: 0.9183


saving the model

In [16]:

import tensorflow as tf

from keras.models import load_model

model.save('model_vgg16.h5')

In [17]:



model = load_model('model_vgg16.h5')

prediction 

In [18]:

#selecting an image from validation dataset for prediction

img = image.load_img('../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/person1949_bacteria_4880.jpeg',target_size=[224,224])

In [19]:
x = image.img_to_array(img)

In [20]:
x = np.expand_dims(x,axis=0)

In [21]:
img_data= preprocess_input(x)

In [22]:
classes = model.predict(img_data)

In [23]:
# 0 refers not effected by pneumonia
# 1 refers effected with pneumonia

print(classes)

[[0. 1.]]
